In [18]:
import io
import os
import signal
import RPi.GPIO as GPIO
import time
import subprocess
import picamera
import numpy as np

from google.cloud import vision
from google.cloud.vision import types

TRIG = 11
ECHO = 12
MIN_V = 45
MAX_V = 70
MIN_T = 0.00011764705
MAX_T = 0.02352941176
TMP_FILE = 'resources/tmp.jpg'
FS = 5000       # sampling rate, Hz, must be integer
SENSE_DUR = 2   # no idea what unit this is in, may be float
SPEAK_VOL = 70
IMG_CYCLE = 20

def set_volume(percent):
    """OS call to set volume"""
    subprocess.call(['amixer', 'set', 'PCM', str.format('{}%', percent)])

def speak(string):
    """External voice program process"""
    set_volume(SPEAK_VOL)
    subprocess.call(['espeak', string])
    
# Part II: Regarding ultrasonic
def dur_to_vol(t):
    return MAX_V - ((MAX_V - MIN_V) / (MAX_T - MIN_T))*t

def sound_proc(freq):
    return subprocess.Popen(['speaker-test', '--frequency', str(freq), '--test', 'sine'])

def duration():
    """Calculated with sonic sensor"""
    GPIO.output(TRIG, 0)
    time.sleep(0.000002)
    GPIO.output(TRIG, 1)
    time.sleep(0.00001)
    GPIO.output(TRIG, 0)
    while GPIO.input(ECHO) == 0: pass
    time1 = time.time()
    while GPIO.input(ECHO) == 1: pass
    time2 = time.time()
    t = time2 - time1
    if (MIN_T < t < MAX_T):
        return t

# Part III: Regarding color
def avg_color(rgb_array):
    return np.mean(rgb_array, axis=(0, 1)).astype(int)

def color_to_freq(c):
    r, g, b = (((c[k]) >> 4) for k in range(3))  # trim 4 bit from each value
    return ((r << 8) + (g << 4) + b)

def main():
    with picamera.PiCamera() as camera:
        camera.resolution = (320, 240)  # Scale down resolution
        try:
            # Google Vision setup
            os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "api-key.json"
            client = vision.ImageAnnotatorClient()  # Instantiates a client

            # Sonic setup
            GPIO.setmode(GPIO.BOARD)
            GPIO.setup(TRIG, GPIO.OUT)
            GPIO.setup(ECHO, GPIO.IN)

            # Color setup
            rgb_array = np.empty((240, 320, 3), dtype=np.uint8)

            # Turn on the sound
            set_volume(MIN_V)
            proc = sound_proc(440)

            counter = 0  # To keep count for Google Vision
            while True:
                counter += 1
                
                # Image processing (rare)
                if counter == IMG_CYCLE:
                    counter = 0

                    # Capture the image to file...
                    camera.capture(TMP_FILE)
                    # ... just to load it back to memory
                    with io.open(TMP_FILE, 'rb') as image_file:
                        content = image_file.read()
                    jpg = types.Image(content=content)

                    # Process the image on the cloud
                    response = client.label_detection(image=jpg)
                    labels = response.label_annotations

                    for label in labels[:2]:
                        print(label.description, end=' ')
                        proc.kill()
                        speak(label.description)

                # Audio sensing (common)
                else:
                    # Sonic part
                    t = duration()  # Gather distance
                    if t is not None:
                        volume = dur_to_vol(t)
                        print("{:.2f}".format(volume), end=' ')
                    else:
                        volume = MIN_V
                    set_volume(volume)

                    # Color:
                    camera.capture(rgb_array, 'rgb')
                    freq = color_to_freq(avg_color(rgb_array))
                    print(freq, end=' ')
                    
                    # Play the sound
                    try:
                        proc.kill()  # Kill the old one first
                    except e:
                        print(e)
                    proc = sound_proc(freq)

        except Exception as e:
            print(e)
        finally:
            GPIO.cleanup()
            proc.kill()
            
main()

1313 69.73 1586 69.70 1587 1894 69.05 1894 1893 68.67 2167 68.14 1912 67.50 2184 1911 58.10 1895 1894 62.57 1365 69.27 1074 69.68 1058 58.33 1911 1894 62.72 1621 69.52 1057 floor material 1621 1621 63.39 1604 69.70 1893 69.26 1603 1620 1876 1621 63.20 1894 69.35 1876 69.69 1604 2167 1876 69.59 1603 1348 69.70 1893 69.71 1331 1912 1910 floor light 54.12 1912 54.20 1911 54.20 1911 54.17 1911 54.16 1911 54.18 1655 54.21 1639 54.18 1895 54.18 1638 54.18 1638 69.81 1638 69.82 1638 54.16 1894 54.15 1638 54.13 1638 54.15 802 54.07 1075 54.17 1074 54.18 1346 yellow close up 54.11 1638 54.13 

KeyboardInterrupt: 